# Riammissione Pazienti Diabetici

Il numero di riammissioni ospedaliere per alcune patologie viene considerato un importante indicatore di qualità di un ospedale, oltre ad impattare in modo pesante sul budget destinato alla sanità.

A causa della privacy trovare un buon dataset è difficile: in assenza di dati reali ci siamo basati su un dataset pubblico, reperibile al link https://archive.ics.uci.edu/ml/datasets/diabetes+130-us+hospitals+for+years+1999-2008# e che copre i dati di ricovero relativi a pazienti diabetici in 130 ospedali statunitensi, durante un periodo di 10 anni (1999 - 2008).

### Tipi di feature nel dataset
Sono registrati 101766 ricoveri unici, fatti da 71518 pazienti differenti, con feature di diverso tipo: 
* dati anagrafici (sesso, fascia di età, peso)
* tipo di ammissione / dimissione / tempo di permanenza in ospedale
* reparto di ammissione
* numero di esami / operazioni / medicazioni effettuati sul paziente
* numero di visite al pronto soccorso / visite specialistiche / ricoveri nell'anno precedente
* numero di diagnosi effettuate sul paziente durante il ricovero
* codice identificativo delle prime tre diagnosi secondo il codice ICD https://en.wikipedia.org/wiki/International_Classification_of_Diseases 
* tipo di reparto dove è avvenuto il ricovero
* terapia farmacologica, viene indicato se un farmaco è stato usato e come il dosaggio è cambiato nel tempo (stabile, crescente, decrescente)
* cambio di terapia diabetica (dosaggio o farmaco)
* utilizzo di terapia diabetica
* valori glicemia nel sangue, divisi in range ( > 200, > 300, Normale) ed eventualmente "None" se l'esame non è stato effettuato
* valori di emoglobina glicata (A1C) al momento dell'ammissione, divisi per range ( > 7 %, > 8 %, Normale) ed eventualmente "None" se l'esame non è stato effettuato

**Il campo più importante è quello che indica se il paziente è stato nuovamente ricoverato entro 30 giorni, oltre i 30 giorni o se non è mai stato riammesso in ospedale: sarà oggetto del nostro modello predittivo.**

In [16]:
# INSTALLO NELL'AMBIENTE LE LIBRERIE NECESSARIE PER FAR FUNZIONARE IL CODICE

!pip install pandas
!pip install IPython

In [17]:
# CARICO LE LIBRERIE NECESSARIE PER FAR FUNZIONARE IL CODICE, E IL DATASET UTILIZZATO NELLA DEMO
import pandas as pd
from IPython.display import display, HTML

original = pd.read_csv('demo_dataset.csv')

In [21]:
# UTILIZZIAMO UN ESTRATTO DEL DATASET DEMO PER VISUALIZZARE I DATI "NUDI E CRUDI": LE CONSIDERAZIONI CHE FAREMO SONO QUELLE CHE SONO STATE UTILIZZATE PER CREARE IL DATASET "PULITO" SU CUI ABBIAMO FATTO IL TRAINING DEL MODELLO
samples = original.sample(20)
display(HTML(samples.to_html()))

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
919,124483536,3877371,Caucasian,Female,[80-90),?,6,22,17,3,?,?,51,3,14,0,0,0,812,424,599,9,None,None,No,No,No,No,No,No,No,Steady,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,Ch,Yes,NO
689,77238636,1534050,Caucasian,Female,[60-70),?,3,1,4,5,?,?,55,5,17,0,0,0,410,414,272,6,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
739,47689278,4984353,Asian,Female,[10-20),?,1,1,7,2,?,Pediatrics-CriticalCare,71,0,8,0,0,0,250.13,276,?,2,None,>8,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,NO
762,36873264,61782138,Caucasian,Male,[60-70),?,6,1,17,2,?,Cardiology,20,3,12,0,0,0,428,425,416,8,None,>8,No,No,No,No,No,No,No,Up,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Ch,Yes,>30
120,165337446,23789313,AfricanAmerican,Female,[50-60),?,1,1,7,6,BC,Family/GeneralPractice,34,3,24,0,0,0,562,285,250.02,8,None,None,Steady,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,>30
496,124751994,80841483,Caucasian,Male,[50-60),?,1,1,7,4,?,InternalMedicine,58,2,26,0,0,1,536,250.92,786,9,None,>8,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,NO
616,170942700,23720346,Caucasian,Female,[80-90),?,1,3,7,6,MC,Orthopedics-Reconstructive,50,3,37,0,0,0,733,427,428,9,None,None,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
929,121738668,41955165,Caucasian,Male,[40-50),?,3,1,1,12,MD,?,1,0,15,0,0,0,296,465,344,8,None,None,Steady,No,No,No,No,No,Up,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,Ch,Yes,NO
614,64256556,23584023,Caucasian,Female,[70-80),?,5,6,17,4,?,?,16,0,15,2,0,1,250.8,424,397,9,Norm,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,>30
269,161619048,40473621,Caucasian,Male,[70-80),?,1,1,7,4,MC,?,41,0,16,0,0,2,434,433,238,7,None,None,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,>30


### Preprocessing dei dati
Alla base di un buon modello di ML si trovano dei dati puliti, il che significa levare le righe con poche features valorizzate, rimuovere colonne in cui la maggior parte dei dati è mancante, oppure utilizzare degli aggregati al posto dei dati "nudi e crudi".

In [ ]:
# GLI ID UTENTE CI AIUTERANNO A TROVARE I DATI CORRISPONDENTI NEL DATASET PULITO
ids = samples['patient_nbr'].unique()  

In [ ]:
cleaned = pd.read_csv('demo_dataset_cleaned.csv')

In [10]:
X = original.sample(10)
ids = X['patient_nbr'].unique()
cleaned_X = cleaned.loc[cleaned['patient_nbr'].isin(ids)]

In [11]:
data = cleaned_X.drop(['readmitted', 'patient_nbr', 'encounter_id'], axis = 1)

In [12]:
scoring_uri = "http://51219778-b190-4555-8416-bb15bcba4112.westeurope.azurecontainer.io/score"
import requests
import json

scoring_dataset = data.values.tolist()
# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": scoring_dataset})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(scoring_uri , input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(scoring_dataset)):
    print ("Patient {}".format([i]), predicted_classes[i] )

Patient [0] not readmitted
Patient [1] not readmitted
Patient [2] not readmitted
Patient [3] not readmitted
Patient [4] not readmitted
Patient [5] not readmitted
Patient [6] not readmitted
Patient [7] not readmitted
Patient [8] not readmitted
Patient [9] not readmitted
